# Table 7.6

- Add 5% to fixed-SST ERF for surface cooling

In [1]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as pl
import pandas as pd
from matplotlib import gridspec, rc
from matplotlib.lines import Line2D
from netCDF4 import Dataset
import warnings

from ar6.constants import NINETY_TO_ONESIGMA

In [2]:
# # # add models from Zelinka 14
z14_data = np.array([
    [-0.66, 0.28, -0.24, -0.01, 0.15, 0.01, -0.22], #'IPSL-CM5A-LR'
    [-0.58, 0.17, -0.51, 0.01, -0.01, 0.11, -0.04], #'CanESM2'
    [-0.63, 0.29, -0.80, 0.03, 0.18, 0.13, -0.17], #'NorESM1-M'
    [-1.13, 0.49, -0.76, 0.08, 0.00, -0.02, -0.21], #'CSIRO-Mk3-6-0'
    [-0.53, 0.26, -1.00, 0.06, -0.13, 0.14, -0.05], #'HadGEM2-A'
    [-0.91, 0.41, -0.99, -0.03, -0.13, 0.11, 0.02], #'GFDL-CM3'
    [-0.66, 0.16, -0.93, -0.01, -0.28, 0.22, 0.27], #'MIROC5'
    [-0.11, 0.12, -1.77, -0.09, -0.23, 0.00, 0.95], #'MRI-CGCM3'
    [-0.33, 0.21, -1.93, 0.06, -0.11, 0.17, 0.57], #'CESM1-CAM5'
])
print(z14_data[:,0])
ERFariCMIP5 = z14_data[:,0]+z14_data[:,1]+z14_data[:,5]
ERFaciCMIP5 = z14_data[:,2]+z14_data[:,3]+z14_data[:,4]+z14_data[:,6]
ERFCMIP5 = ERFariCMIP5+ERFaciCMIP5
print(ERFCMIP5)
# for row in z14_data:
#     if row[0] not in models:
#         models=models+[row[0]]
#     ERFariSW_sca_em[row[0]] = np.float64(row[1])
#     ERFariSW_abs_em[row[0]] = np.float64(row[2])
#     ERFaciSW_sca_em[row[0]] = np.float64(row[3])
#     ERFaciSW_abs_em[row[0]] = np.float64(row[4])
#     ERFaciSW_clt_em[row[0]] = np.float64(row[5])
#     ERFariLW_em[row[0]]     = np.float64(row[6])
#     ERFaciLW_em[row[0]]     = np.float64(row[7])

[-0.66 -0.58 -0.63 -1.13 -0.53 -0.91 -0.66 -0.11 -0.33]
[-0.69 -0.85 -0.97 -1.55 -1.25 -1.52 -1.23 -1.13 -1.36]


In [3]:
df = pd.read_csv('../data_input/Smith_et_al_ACP_2020/aprp_aer.csv')
# treat GISS p1 and GISS p3 as separate models
df.loc[(df['Model']=='GISS-E2-1-G') & (df['Run']=='r1i1p1f1'),'Model'] = 'GISS-E2-1-G p1'
df.loc[(df['Model']=='GISS-E2-1-G') & (df['Run']=='r1i1p1f1'),'Model'] = 'GISS-E2-1-G p3'
#df.set_index('Model', inplace=True)
df

,Model,ERF,ERF_LW,ERF_SW,ERFaci,ERFaciLW,ERFaciSW,ERFaciSW_abs,ERFaciSW_clt,ERFaciSW_sca,...,ERFariaciLW,ERFariaciSW,Run,ghanERFaci,ghanERFaciLW,ghanERFaciSW,ghanERFari,ghanERFariLW,ghanERFariSW,od550aer
0,ACCESS-CM2,-1.093110,0.302125,-1.395235,-0.889889,0.041806,-0.931695,-0.008777,-0.124594,-0.798324,...,0.302125,-1.416070,r1i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CanESM5,-0.851293,-0.019024,-0.832269,-1.040365,-0.078374,-0.961992,0.057386,-0.136031,-0.883347,...,-0.019024,-1.042312,r1i1p2f1,-1.273250,-0.048576,-1.224674,0.361977,0.035551,0.326426,NaN
2,CESM2,-1.371578,0.155730,-1.527308,-1.573986,0.103266,-1.677252,0.030856,-0.006556,-1.701553,...,0.155730,-1.583392,r1i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,0.012937
3,CNRM-CM6-1,-1.154830,0.092881,-1.247711,-0.819226,-0.052629,-0.766596,-0.052504,0.054585,-0.768677,...,0.092881,-1.174648,r1i1p1f2,-0.790306,-0.051302,-0.739004,-0.421651,0.002796,-0.424447,0.021007
4,CNRM-ESM2-1,-0.740564,0.080886,-0.821450,-0.606682,-0.019046,-0.587637,-0.042614,-0.027340,-0.517683,...,0.080886,-0.829367,r1i1p1f2,-0.607232,-0.016657,-0.590575,-0.213161,0.004485,-0.217646,0.016143
5,EC-Earth3,-0.798890,0.043045,-0.841935,-0.479353,-0.018497,-0.460855,-0.037547,-0.079197,-0.344111,...,0.043045,-0.892860,r1i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,GFDL-CM4,-0.731706,0.066934,-0.798641,-0.683897,-0.059019,-0.624878,0.000186,-0.085712,-0.539351,...,0.066934,-0.863261,r1i1p1f1,-0.807595,-0.044227,-0.763368,0.075006,0.017284,0.057721,0.039799
7,GFDL-ESM4,-0.697921,-0.109431,-0.588490,-0.803989,-0.169932,-0.634056,0.008705,-0.055594,-0.587167,...,-0.109431,-0.749773,r1i1p1f1,-0.923602,-0.153508,-0.770094,0.256108,0.020959,0.235149,0.040951
8,GISS-E2-1-G p1,-1.318632,0.291122,-1.609753,-0.770535,0.096234,-0.866769,0.013002,-0.935966,0.056195,...,0.291122,-1.584448,r1i1p1f1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,GISS-E2-1-G,-0.927439,0.131050,-1.058490,-0.367913,-0.052330,-0.315582,-0.007288,-0.061774,-0.246520,...,0.131050,-1.106024,r1i1p3f1,NaN,NaN,NaN,NaN,NaN,NaN,0.046236


In [4]:
data = []
erfari = []
erfaci = []
total = []

for model in df.Model.unique():
    erfari.append(1.05 * df.ERFari[df.Model==model].mean())
    erfaci.append(1.05 * df.ERFaci[df.Model==model].mean())
    total.append(1.05 * df.ERFari[df.Model==model].mean() + 1.05 * df.ERFaci[df.Model==model].mean())
    data.append([model, erfari[-1], erfaci[-1], total[-1]])
erfari = np.array(erfari)
erfaci = np.array(erfaci)
total = np.array(total)
data.append(['CMIP6 mean', np.nanmean(erfari), np.nanmean(erfaci), np.nanmean(total)])
data.append(
    [
        'CMIP6 range',
        NINETY_TO_ONESIGMA * np.nanstd(erfari, ddof=1), 
        NINETY_TO_ONESIGMA * np.nanstd(erfaci, ddof=1),
        NINETY_TO_ONESIGMA * np.nanstd(total, ddof=1)
    ]
) # use sample stdev and inflate to 5-95%
data.append(['CMIP5 mean', np.mean(1.05 * ERFariCMIP5), np.mean(1.05 * ERFaciCMIP5), np.mean(1.05 * ERFCMIP5)])
data.append(
    [
        'CMIP5 range',
        NINETY_TO_ONESIGMA * np.nanstd(1.05 * ERFariCMIP5),
        NINETY_TO_ONESIGMA * np.nanstd(1.05 * ERFaciCMIP5),
        NINETY_TO_ONESIGMA * np.nanstd(1.05 * ERFCMIP5)
    ]
)

In [5]:
df = pd.DataFrame(data, columns = ['Model', 'ERFari', 'ERFaci', 'ERFari+aci'])

In [6]:
df.to_csv('../data_output/table7.6.csv', index=False)